In [14]:
import pytest
import sys
import os 
import pandas as pd
import importlib
%matplotlib inline
sys.path.append('/workspace')
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  

In [15]:
from modules.Pipelines.FlowsPipe import FlowsPipe
from modules.Pipelines.ERCsAggregatedPipe import ERCsAggregatedPipe
from modules.Transformers.Aggregators.NetworkDataMetricsAggregator import NetworkDataMetricsAggregator
from modules.Transformers.Aggregators.NetworkDataXDayAverage import NetworkDataXDayAverage
from modules.Transformers.PercentGrowth import PercentGrowth
from modules.Visualizers.ERCs.LineChartERCsAggregateMetric import LineChartERCsAggregateMetric
from modules.Visualizers.Table import Table
from modules.Visualizers.LineChart import LineChart
from modules.Visualizers.LineChartMetricByAssets import LineChartMetricByAssets
from modules.Visualizers.MarketShareAssetsByMetric import MarketShareAssetsByMetric
from modules.Transformers.Rankers.RankMetricByAssets import RankMetricByAssets

In [16]:
api_key = 'KKzV6V2DTY87v3m1dGZu'
stablecoins = ['dai', 'gusd', 'tusd', 'usdc', 'pax', 'usdt_eth']
start = '2019-01-01'
metrics = ['AdrActCnt', 'AdrBal1in1BCnt','AdrBalUSD10Cnt','CapMrktCurUSD', 'CapRealUSD','IssTotNtv', 'TxCnt', 'TxTfrCnt', 'TxTfrValAdjUSD', 'VtyDayRet30d']
pipe = ERCsAggregatedPipe(api_key=api_key, ERCs=None, metrics=metrics, start=start, aggregate_ERCs=False)
df = pipe.run_pipeline().get_data()

In [21]:
table_df = NetworkDataXDayAverage(number_of_days=30).transform(df)
ranking_metrics = ['TxCnt', 'TxTfrCnt', 'TxTfrValAdjUSD', 'CapRealUSD', 'CapMrktCurUSD']
for ranking_metric in ranking_metrics:
    ranked = RankMetricByAssets(metric=ranking_metric).transform(table_df)
    top_10 = ranked.loc[0:9]
    top_10_assets = list(top_10['asset'])
    top_10_pipe = ERCsAggregatedPipe(api_key=api_key, ERCs=top_10_assets, metrics=metrics, start=start, aggregate_ERCs=False)
    top_10_df = pipe.run_pipeline().get_data()
#     top_10_table = NetworkDataXDayAverage(number_of_days=30).transform(top_10_df)
    market_share_chart = MarketShareAssetsByMetric(title='{} Market Share, 2019'.format(ranking_metric), metric=ranking_metric, assets=top_10_assets).create_visual(top_10_df)
    market_share_chart.fig.show()


In [18]:
market_share_chart = MarketShareAssetsByMetric(title='Transfer Market Share', metric='TxTfrCnt', assets=top_10_assets).create_visual(top_10_df)
market_share_chart.fig.show()

In [19]:
test_chart = LineChartMetricByAssets(title='ranked', metric=ranking_metric, assets=top_10_assets[1:]).create_visual(growth_df)
test_chart.fig.show()

NameError: name 'growth_df' is not defined